In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.fftpack import fft,ifft
import pandas as pd

from scipy.optimize import minimize
from scipy.optimize import basinhopping
from scipy.optimize import fmin_bfgs

from pprint import pprint
from iminuit import Minuit

In [2]:
# N=20取bin的函数

def databin_20(lst):
    result = [[]]    
    length = len(lst)
    n = 0
    for i in range(length):
        result[-1].append(math.log(lst[i],10))
        n = n+1
        if n == 20:
            n = 0
            result.append([])
    output=[]
    for j in range(len(result)):
        output.append(np.mean(result[j])) 
    return output

def databin_20_std(lst):
    result = [[]]
    length = len(lst)
    n = 0
    for i in range(length):
        result[-1].append(math.log(lst[i],10))
        n = n+1
        if n == 20:
            n = 0
            result.append([])        
    output=[]
    for j in range(len(result)):
        output.append(np.std(result[j]))
    return output


In [3]:
def chi2_of_model(x0):
    A,f_b,alpha_H=x0
    
    data = pd.read_csv("0.3_10_final.csv")  
    counts_data = data['RATE']
    dt=100
    mean_x_data = np.mean(counts_data)
    N=len(counts_data)
    N_randomlc=len(counts_data)
    
    omega = []
    POW = []
    DFT = []
    fr = []
    fi = []
    f1 = []
    f2 = []
    p = []
    f = []
    
    counts_list=[None for i in range(100)]
    
    
    # 500条光变曲线
    for a in range(100): 
        f_b=1.7E-4
        alpha_H=3.8
        alpha_L=1.0
        for j in range(1,int(N_randomlc)+1):
            omega.append(j/(N_randomlc*dt))
            POW.append(((omega[-1]**(-alpha_L))/(1+(omega[-1]/f_b)**(alpha_H-alpha_L)))*A)
            DFT.append(complex(np.sqrt(POW[-1]),np.sqrt(POW[-1])))
            s1=np.random.normal(loc=0.0, scale=1.0, size=None)
            s2=np.random.normal(loc=0.0, scale=1.0, size=None)
            fr.append((DFT[-1].real)*s1)
            fi.append((DFT[-1].imag)*s2)
            f1.append(complex(fr[-1],fi[-1]))
        counts = ifft(f1)
        counts_list[a]=counts
    
    
    #周期图
    per_list=[None for i in range(100)]
    for a in range(100):
        nf = N/2 
        df = 1/(dt*N)
        F_a = np.arange(1,nf+1)
        F = [i*df for i in F_a]
        F1 = F[0:int(nf)]
        mean_x = np.mean(counts_list[a])
        x_new  = [i-mean_x for i in counts_list[a]]
        dft   = fft(counts_list[a])
        dft_1 = dft[1:int(nf)+1]
        per = (abs(dft_1)**2)
        per_list[a] = per
    
    
    # 500条周期图取 mean,std
    per_everypoint_list=[]
    for i in range(int(nf)):
        per_everypoint_list.append([])
    ADM=[]
    ADM_std=[]
    for m in range(int(nf)):
        for n in range(100):
            per_everypoint_list[m].append(per_list[n][m])
        
    for m in range(int(nf)):
        ADM.append(np.mean(per_everypoint_list[m]))
        ADM_std.append(np.std(per_everypoint_list[m]))    
        
        
    POW1=POW[0:int(nf)]
    P_TIMES_F = np.multiply(np.array(F1),np.array(POW1))
    F1_log = [math.log(i,10) for i in F1]
    PTF_log = [math.log(i,10) for i in P_TIMES_F]
    

    
    
    #加上泊松项并分bin
    ADM_with_poisson = [i+2/mean_x_data for i in ADM]
    ADM_binned_ps = databin_20(ADM_with_poisson)
    F1_binned = databin_20(F1)
    ADM_times_f_b_ps = np.array(F1_binned)+np.array(ADM_binned_ps)
    
    
    # 从数据得到参考周期图
    pnum = np.arange(len(counts_data))
    t = [i*dt for i in pnum]
    
    nf = N/2
    df = 1/(dt*N)
    F_num = np.arange(1,nf)
    F = [i*df for i in F_num]
    mean_x = np.mean(counts_data)
    dft = fft(counts_data)
    dft1= dft[1:int(nf)+1]
    per_data = (abs(dft1)**2)*2*dt/((mean_x**2)*N)
    
    
    # 数据分bin
    per_data_binned = databin_20(per_data)
    per_data_b_std = databin_20_std(per_data)
    p_times_f_data_b = np.array(F1_binned)+np.array(per_data_binned)
    
    
    chi2=0
    for i in range(len(ADM_binned_ps)):
        chi2 = chi2+(((ADM_binned_ps[i]-per_data_binned[i])/per_data_b_std[i])**2)    
    
    print('chi2 of this time = ',chi2)
    
    return (chi2)

In [ ]:
bnds=(0.001,0.01),(1.0E-4,1.0E-3),(3.0,5.0)
minimizer_kwargs = {"bounds":(bnds)}  # minimizer_kwargs=minimizer_kwargs
x0=[0.005,1.65E-4,3.8]
x0_array=np.array(x0)
m = basinhopping(chi2_of_model,x0_array,minimizer_kwargs=minimizer_kwargs)

chi2 of this time =  2.5647157804877465
chi2 of this time =  2.356916639654567
chi2 of this time =  2.2804357433108695
chi2 of this time =  2.4561180148891824
chi2 of this time =  5.2919623187597535
chi2 of this time =  6.988286493885878
chi2 of this time =  7.65212686146988
chi2 of this time =  5.887300541983052
chi2 of this time =  4.497063717434185
chi2 of this time =  5.088383077968453
chi2 of this time =  5.909629153396874
chi2 of this time =  4.170673012456146
chi2 of this time =  3.0827568795404057
chi2 of this time =  3.5949966542017857
chi2 of this time =  3.46837711562632
chi2 of this time =  2.516853788346765
chi2 of this time =  2.402054233718307
chi2 of this time =  2.9247712702991997
chi2 of this time =  2.2766751782333774
chi2 of this time =  2.3804523546302963
chi2 of this time =  2.2040590088152205
chi2 of this time =  2.613701054166237
chi2 of this time =  2.8230470419984557
chi2 of this time =  3.2733305870744887
chi2 of this time =  2.4752542605375716
chi2 of this t

chi2 of this time =  5.532024664225293
chi2 of this time =  5.129520721006119
chi2 of this time =  4.760152471244487
chi2 of this time =  4.320614885396767
chi2 of this time =  4.327252465540671
chi2 of this time =  4.578468757244823
chi2 of this time =  4.282085086923493
chi2 of this time =  4.11747017400065
chi2 of this time =  4.751545897262618
chi2 of this time =  4.663801368546676
chi2 of this time =  5.179650554459821
chi2 of this time =  6.127833321464758
chi2 of this time =  6.173354374294328
chi2 of this time =  5.261671498427361
chi2 of this time =  5.295211723233065
chi2 of this time =  5.568834986158912
chi2 of this time =  3.4284982539583786
chi2 of this time =  4.981216275879122
chi2 of this time =  5.9224644021906325
chi2 of this time =  6.23040151698111
chi2 of this time =  4.872480281281017
chi2 of this time =  5.12175991171697
chi2 of this time =  4.232491111180668
chi2 of this time =  5.151365910627458
chi2 of this time =  5.063521973576354
chi2 of this time =  5.168

chi2 of this time =  4.333821706302005
chi2 of this time =  6.4463626138952295
chi2 of this time =  5.851285617522449
chi2 of this time =  5.714599826991227
chi2 of this time =  7.067626871351677
chi2 of this time =  2.5626927049455768
chi2 of this time =  2.66299838632455
chi2 of this time =  4.100300772479146
chi2 of this time =  2.955341916687028
chi2 of this time =  2.1484046809058297
chi2 of this time =  2.031308246884903
chi2 of this time =  2.008796000192991
chi2 of this time =  1.8770593227166394
chi2 of this time =  2.4900575989373945
chi2 of this time =  2.0922096194510993
chi2 of this time =  3.3266044944610758
chi2 of this time =  3.583848766843573
chi2 of this time =  3.020747794334543
chi2 of this time =  3.0961164080259818
chi2 of this time =  4.380212445885949
chi2 of this time =  2.824846775164998
chi2 of this time =  4.540193903221004
chi2 of this time =  4.643768175443418
chi2 of this time =  4.01828509207059
chi2 of this time =  3.2021576782201406
chi2 of this time 

chi2 of this time =  8.578836405403539
chi2 of this time =  7.245390469399749
chi2 of this time =  5.747749940692232
chi2 of this time =  6.377338123239926
chi2 of this time =  6.446311624432629
chi2 of this time =  5.742170167699179
chi2 of this time =  6.156407450461948
chi2 of this time =  7.3026245150998275
chi2 of this time =  4.450333177311715
chi2 of this time =  6.678771440194751
chi2 of this time =  7.098665866379458
chi2 of this time =  4.999965744437984
chi2 of this time =  8.155470451582998
chi2 of this time =  6.482681547027208
chi2 of this time =  7.335997289173824
chi2 of this time =  8.452839350850644
chi2 of this time =  8.282333059865422
chi2 of this time =  6.210967667403535
chi2 of this time =  7.368807132882061
chi2 of this time =  7.444185950968333
chi2 of this time =  6.037233406136625
chi2 of this time =  5.74641545019105
chi2 of this time =  6.745988502030016
chi2 of this time =  6.61906330227801
chi2 of this time =  7.799544980031599
chi2 of this time =  7.641

chi2 of this time =  7.5774254256133595
chi2 of this time =  5.095016459264636
chi2 of this time =  6.529175285687497
chi2 of this time =  5.283993273521374
chi2 of this time =  5.558261793627328
chi2 of this time =  6.361202685658857
chi2 of this time =  5.826056028881989
chi2 of this time =  6.793622642746883
chi2 of this time =  6.790576340362525
chi2 of this time =  6.230409003173833
chi2 of this time =  7.7600349498304855
chi2 of this time =  6.258731765380898
chi2 of this time =  5.705185137570946
chi2 of this time =  7.915522583884037
chi2 of this time =  6.894934632534232
chi2 of this time =  8.452233238591246
chi2 of this time =  7.038316659798138
chi2 of this time =  7.019052226305582
chi2 of this time =  5.219141539840213
chi2 of this time =  6.4641589656287834
chi2 of this time =  6.044742527657093
chi2 of this time =  7.910859895543377
chi2 of this time =  5.545572750521526
chi2 of this time =  6.915253267673182
chi2 of this time =  9.54389609037552
chi2 of this time =  6.

chi2 of this time =  4.861422034114517
chi2 of this time =  8.443450453479429
chi2 of this time =  5.037778843477571
chi2 of this time =  7.138285079997382
chi2 of this time =  6.86264899851411
chi2 of this time =  7.76317778216154
chi2 of this time =  7.974124358447988
chi2 of this time =  6.665697197652338
chi2 of this time =  6.734720330485849
chi2 of this time =  5.244200835256206
chi2 of this time =  6.910843711855031
chi2 of this time =  6.476312897870147
chi2 of this time =  7.327795226593349
chi2 of this time =  4.285221785150986
chi2 of this time =  5.976816114861795
chi2 of this time =  6.331476579378532
chi2 of this time =  5.625115319365337
chi2 of this time =  5.652178405328681
chi2 of this time =  6.427621790919019
chi2 of this time =  7.328177061761224
chi2 of this time =  6.165980141075869
chi2 of this time =  4.204321264954085
chi2 of this time =  6.089111446351841
chi2 of this time =  7.769198705733584
chi2 of this time =  5.219491734404652
chi2 of this time =  5.6992

chi2 of this time =  7.201879941709578
chi2 of this time =  7.230509053631198
chi2 of this time =  7.647276747261727
chi2 of this time =  6.749175967892233
chi2 of this time =  5.578920612128485
chi2 of this time =  7.870105922088496
chi2 of this time =  8.136243394115528
chi2 of this time =  7.496565106233881
chi2 of this time =  6.043895119445886
chi2 of this time =  7.83488461513202
chi2 of this time =  6.916528844452033
chi2 of this time =  8.410174801071797
chi2 of this time =  7.071636885408332
chi2 of this time =  8.073469105989941
chi2 of this time =  6.493440917297542
chi2 of this time =  7.201905915645837
chi2 of this time =  7.922479677136244
chi2 of this time =  6.147937122315349
chi2 of this time =  5.693620073147212
chi2 of this time =  5.722617782883543
chi2 of this time =  7.1455024406845915
chi2 of this time =  7.04273062980403
chi2 of this time =  6.855088830947068
chi2 of this time =  6.30475453104464
chi2 of this time =  6.26572397792869
chi2 of this time =  6.14194

chi2 of this time =  5.511545679758674
chi2 of this time =  7.040432154820932
chi2 of this time =  7.448557346897108
chi2 of this time =  5.934784017270087
chi2 of this time =  7.426302110697071
chi2 of this time =  5.208448679596522
chi2 of this time =  7.462477500889771
chi2 of this time =  6.224632547901949
chi2 of this time =  6.441398920327682
chi2 of this time =  7.77839156721338
chi2 of this time =  6.083479190408861
chi2 of this time =  7.4975365404476415
chi2 of this time =  6.6108956683033835
chi2 of this time =  7.061569946844459
chi2 of this time =  7.496542610529686
chi2 of this time =  7.732044259446081
chi2 of this time =  5.402831207373088
chi2 of this time =  6.468758625203779
chi2 of this time =  6.983943906987246
chi2 of this time =  7.76797220642121
chi2 of this time =  5.434018489340646
chi2 of this time =  6.368755721729132
chi2 of this time =  6.509306849148833
chi2 of this time =  6.317655286935961
chi2 of this time =  8.431542377288277
chi2 of this time =  5.87

chi2 of this time =  8.35617388350133
chi2 of this time =  5.049751043846779
chi2 of this time =  7.127321348785999
chi2 of this time =  6.7640159448006445
chi2 of this time =  5.261768175374466
chi2 of this time =  5.125155119585408
chi2 of this time =  5.427545250197181
chi2 of this time =  6.133109035313535
chi2 of this time =  8.164410212418916
chi2 of this time =  4.874935282322292
chi2 of this time =  7.67004911553316
chi2 of this time =  6.83628557739577
chi2 of this time =  8.066239067188969
chi2 of this time =  9.540459712298896
chi2 of this time =  7.27814594323821
chi2 of this time =  7.7721660814051505
chi2 of this time =  6.737425735994347
chi2 of this time =  6.197202855899826
chi2 of this time =  7.166125592263014
chi2 of this time =  7.290437427629537
chi2 of this time =  5.946663765010245
chi2 of this time =  5.939289726971904
chi2 of this time =  6.60986599964231
chi2 of this time =  4.469002522061975
chi2 of this time =  7.356786161149926
chi2 of this time =  7.39294

chi2 of this time =  4.1196099016820495
chi2 of this time =  5.377181929120236
chi2 of this time =  4.052735152604859
chi2 of this time =  4.6284631142127886
chi2 of this time =  4.0268964986122056
chi2 of this time =  3.7571763518062635
chi2 of this time =  5.263149506692094
chi2 of this time =  4.54241021122165
chi2 of this time =  4.818677380061544
chi2 of this time =  5.246173816564666
chi2 of this time =  4.487791873731124
chi2 of this time =  3.3691937146668254
chi2 of this time =  3.7107823232199646
chi2 of this time =  4.195947872060236
chi2 of this time =  3.7849676990044943
chi2 of this time =  3.5511187509425253
chi2 of this time =  4.545902237368146
chi2 of this time =  4.782692635123882
chi2 of this time =  5.194243310038709
chi2 of this time =  4.892005229945585
chi2 of this time =  5.282371153177919
chi2 of this time =  3.8896729223468465
chi2 of this time =  7.767906550829349
chi2 of this time =  5.051235718179147
chi2 of this time =  3.712050879878469
chi2 of this time

In [ ]:
print(m)